# 04 — Presentation Assets

Generate presentation-ready tables and charts for Step 4.

**Inputs**
- `data/features/customer_features.parquet`
- `data/segments/segment_assignments.parquet`

**Outputs**
- `artifacts/step4_presentation/segment_summary.csv`
- `artifacts/step4_presentation/segment_feature_heatmap.png`
- `artifacts/step4_presentation/segment_sizes.png`
- `artifacts/step4_presentation/segment_feature_heatmap.svg`
- `artifacts/step4_presentation/segment_sizes.svg`


In [ ]:
from pathlib import Path

import pandas as pd

features_path = Path('data/features/customer_features.parquet')
segments_path = Path('data/segments/segment_assignments.parquet')

features = pd.read_parquet(features_path)
segments = pd.read_parquet(segments_path)

data = features.merge(segments, on='user_id', how='inner')
data.head()


In [ ]:
summary = (
    data.groupby('segment')
    .agg(count=('user_id', 'size'))
    .join(data.groupby('segment').mean(numeric_only=True))
    .sort_values('count', ascending=False)
)

output_dir = Path('artifacts/step4_presentation')
output_dir.mkdir(parents=True, exist_ok=True)

summary.to_csv(output_dir / 'segment_summary.csv')
summary.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

feature_cols = summary.select_dtypes(include='number').columns.tolist()
feature_cols = [col for col in feature_cols if col != 'count']

plt.figure(figsize=(12, 6))
sns.heatmap(summary[feature_cols], cmap='viridis', annot=False)
plt.title('Segment Feature Means')
plt.tight_layout()

heatmap_png = output_dir / 'segment_feature_heatmap.png'
heatmap_svg = output_dir / 'segment_feature_heatmap.svg'
plt.savefig(heatmap_png, dpi=300)
plt.savefig(heatmap_svg)
plt.show()


In [ ]:
sizes = summary['count']

plt.figure(figsize=(8, 5))
sizes.plot(kind='bar', color='steelblue')
plt.title('Segment Sizes')
plt.ylabel('Users')
plt.xlabel('Segment')
plt.tight_layout()

sizes_png = output_dir / 'segment_sizes.png'
sizes_svg = output_dir / 'segment_sizes.svg'
plt.savefig(sizes_png, dpi=300)
plt.savefig(sizes_svg)
plt.show()
